In [ ]:
# Connect to MongoDB
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

with open('../data_base_key.txt', 'r') as file:
    file_content = file.read()

uri = file_content

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

## Read Data

In [2]:
db = client['db_products']
collection = db['products']

In [ ]:
for document in collection.find():
    print(document)

In [ ]:
collection.update_many({}, {'$rename': {'Produto': 'product',
                                        'Categoria do Produto': 'product category',
                                        'Preço': 'price',
                                        'Frete': 'shipping fee',
                                        'Data da Compra': 'puchase data',
                                        'Vendedor': 'seller',
                                        'Local da compra': 'place of purchase',
                                        'Avaliação da compra': 'purchase evaluation',
                                        'Tipo de pagamento': 'payment type',
                                        'Quantidade de parcelas': 'number of installments',
                                        'lat': 'latitude',
                                        'lon': 'longitude'}})

In [ ]:
collection.find_one()